In [1]:
from time import time as tictoc
import sys
from sky_patch_class import *
from my_units import * 
from angular_fn import *
from template_fn import *

HomeDir = '../'
DataDir = HomeDir+'data/' ### Set this to the directory where you store your data files
ListDir = HomeDir+'lists/'
ListTauDir = ListDir+'/data_tau/'

Notebook to run the coarse template scan at a scale of approximately beta_t. This notebook should be converted to a python script and run on a cluster, splitting the template scanning locations between multiple cores. Some parameters for the analysis are read in from the command line. For example, the python script to do the template scan on the LMC for beta_t = 0.03 deg should be run as:

python data_template_scan.py LMC 300 40 0

In this case the number of scanning locations are divided in 40 subsets (<= number of cores available) and the 0-th subset is run.

Takes as input:
* data_file_name+'_final.npy', the cleaned data generated in the notebook data_cleaning.ipynb

where data_file_name are 'LMC_disc_5' or 'SMC_disc_4'. Generate as output the tau values (and normalization) at each location:
* data_file_name+'_tau_b'+beta_t_deg+'_'+str(i_step)+'.npy'

where beta_t_deg is the beta_t value used in the analysis given in units of degree/10000 (i.e. 300 for beta_t = 0.03 deg) and i_step is the i-th subset of all the scanning locations.

# Automatic template scan 

## Preamble

In [27]:
### Read in paramters from the command line.
sky_patch_name = sys.argv[1]  # 'LMC' or 'SMC'
beta_t_deg = sys.argv[2]      # beta_t value to use for the analysis (given in units of degree/10000)
n_steps = int(sys.argv[3])    # i.e. 40 (equal or smaller than the number of cores available)
i_step = int(sys.argv[4])     # from 0 to n_steps-1

In [25]:
### Define sky patch to use in the analysis
if sky_patch_name == 'LMC':    
    sky_p = sky_patch(81.28, -69.78, 5*degree, 50*kpc, 'LMC_disc_5', np.array([1.871, 0.391]), pm_esc=0.2, sigma_pm = 0.125)
    print('********** Template analysis on the LMC for beta_t_deg =', beta_t_deg, '**********')
elif sky_patch_name == 'SMC':
    sky_p = sky_patch(12.80, -73.15, 4*degree, 60*kpc, 'SMC_disc_4', np.array([0.686, -1.237]), pm_esc=0.2, sigma_pm = 0.105)
    print('********** Template analysis on the SMC for beta_t_deg =', beta_t_deg, '**********')
else:
    print('ERROR: wrong name provided for the sky patch!')
sys.stdout.flush()

In [51]:
### Parameters for data cleaning
beta_kernel_sub_0 = 0.1*degree; beta_kernel_sub = 0.06*degree;   # gaussian kernels for background subtraction 
n_iter_sub = 3;                                                  # number of iterations for the background subtraction
disc_radius_no_edge = sky_p.disc_radius - beta_kernel_sub_0 - (n_iter_sub+1)*beta_kernel_sub

### Parameters for the template scan
n_betat = 6; ### the template is computed for stars within a circle of radius n_betat*beta_t from the template location
beta_t = float(beta_t_deg)/10000*degree
min_mask = 0.01*degree ### minimum radius of the mask used to compute the template 

In [4]:
### Import the final data
data = np.load(DataDir+sky_p.data_file_name+'_final.npy') 
### np array with n_rows = n_stars and each row must be:
### [ra, dec, proper_motion_weight, weighted_pmra, weighted_pmdec]

## Execution 

In [ ]:
### Determine coordinates for the coarse scanning:
### Pixels within a disc of radius disc_radius_no_edge - n_betat*beta_t from the stellar target center,
### at an angular scale of approximatly beta_t.
scan_coord = fn_coarse_scan_coord(sky_p.disc_center, disc_radius_no_edge, beta_t, n_betat)
n_locations = len(scan_coord[0])
step = math.ceil(n_locations/n_steps)
max_i_step = math.ceil(n_locations/step)
print('Number of template locations: '+str(n_locations)+'. Dividing in '+str(n_steps)+' steps. Number of locations per step: '+str(step)+'.')
sys.stdout.flush()

if i_step >= max_i_step:
    print('The step = '+str(i_step)+' is grater than the maximum allowed step = '+str(max_i_step)+'. Not running the template scan.')
    sys.stdout.flush()
else: 
    print('Running step = '+str(i_step)+'.')
    sys.stdout.flush()
    first_loc = step*i_step; last_loc = min(step*(i_step+1), n_locations)
    
    ### Prepare the data for the template scan at beta_t. Pixelate at angular scale approximatly beta_t/10.
    ### Append the stars pixel number as the first column of the array data
    nside, scan_pix, data = fn_prepare_template_scan(data, [scan_coord[0][first_loc:last_loc], scan_coord[1][first_loc:last_loc]], beta_t)

    tic = tictoc()
    ### Compute the templates
    template_res = fn_template_scan(nside, scan_pix, n_betat, beta_t, data, min_mask)
    ### Save the result of the template scan
    np.save(ListTauDir+sky_p.data_file_name+'_tau_b'+beta_t_deg+'_'+str(i_step), template_res)
    ### The result is saved as a numpy array with n_rows = n_locations and each row is:
    ### [ra, dec, tau_ra, tau_dec, normalization, tau_monopole, normalization_monopole]
    ### The monopole template is computed to have a background-only channel.
    toc = tictoc()    

    print('Template scan completed in', str(toc - tic), 's.')
    sys.stdout.flush()